# Getting Started

LINUX users can open the terminal window with:

In [ ]:
Ctrl+Alt+T

Login into rocket with your account:

In [ ]:
ssh your_account_id@rocket.hpc.ut.ee

### Install and Setting up a conda environment

Steps available at Anaconda website: https://docs.anaconda.com/miniconda/install/#quick-command-line-install

In [ ]:
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm ~/miniconda3/miniconda.sh

In [ ]:
Check you bashrc file (more ~/.bashrc) to see whether conda edited last section of the file

Exit and re-enter terminal

#### Copy the environment and create it in your directory

In [ ]:
conda env create -f /gpfs/helios/projects/echo_workshops/project.1.tk/conda_env/echo_workshop.yml

In [ ]:
conda activate echo_workshop

### Good practices when starting a new project

It is good practice to keep your directory clean and well-organized. 

For example, given that today we will work on the non imputed dataset we can create right away the directory 'non_imputed' with the command `mkdir` (mkdir = make directory), where we will store the dataset and run the analyses.

In [ ]:
mkdir non_imputed

To move into the newly create directory you can use the `cd` command (cd = change directory).

In [ ]:
cd non_imputed

There, you can create right away another directory: `Exploring_PCA`, where you can store the dataset, and run the analyses. 

In [ ]:
mkdir Exploring_PCA 

### Get the dataset
The vcf is readily available at /gpfs/helios/projects/echo_workshops/project.1.tk/data/PopStructureFiles/non_imputed.vcf. Be sure that you are currently in your 'non_imputed/dataset' folder and copy the file in your directory is this way

In [ ]:
cp /gpfs/helios/projects/echo_workshops/project.1.tk/data/PopStructureFiles/non_imputed.vcf .

## Explore the Dataset

We are working on a remote server, where multiple users work with different software needs. Managing many softwares and many versions is complex, and can cause dependency issues. 

Rather than having installed and readily available all softwares we use **modules**. Modules allow to manage and load software environments dynamically, ensuring that the correct versions of software and their dependencies are used without conflicts.

Before **loading** a specific module, we need to search for it, to know whether it is available and under what name. 

`module spider` will provide a list of all available softwares and their version under the searched name. 

`module spider plink`

`module load` is used to activate a specific software environment by loading the desired module. 

`module load plink/1.9-beta6.27`

# Dataset seen through PLINK

In [ ]:
module spider plink
module load plink/1.9-beta6.27

Convert vcf to plink format

In [ ]:
plink --vcf non_imputed.vcf --keep-allele-order --make-bed --out non_imputed

While converting PLINK informs us about:
 * Number of SNPS (variants)
 * Number of individuals (people) and number of male/females
 * Genotyping rate

## Estimating the number of SNPs and samples

We can easily count the number of SNPs by counting the lines in the bim file with bash `wc -l`.

`wc` = word count \
`-l` = lines

In [ ]:
wc -l non_imputed.bim 

Similarly, we can count the number of Samples by counting the lines in the fam file

In [ ]:
wc -l non_imputed.fam

If the FID are available, we can estimate the number of samples per cluster

In [ ]:
awk '{print $1}' non_imputed.fam | sort | uniq -c

## Allele Frequencies
The dataset can be characterized by many rare or many common variants. The allele frequencies will impact the study, and the kind of analyses we can carry on the dataset. For an insight of the allele frequencies in out dataset we can use the `--freq` options in PLINK, which writes a minor allele frequency report to an output **.frq** file.

In [ ]:
plink --bfile non_imputed --freq --out allele_freq_report

And we can see the first few lines with the command `head`

In [ ]:
head allele_freq_report.frq

The file has 6 columns:

* CHR	Chromosome number
* SNP	Variant identifier
* A1	Allele 1 (usually minor)
* A2	Allele 2 (usually major)
* MAF	Allele 1 frequency, stands for Minor Allele Frequency
* NCHROBS	Number of allele observations (number of samples*2)


### How many alleles have a MAF lower than 0.05?

In [ ]:
awk '$5 < 0.05 {print $5}' allele_freq_report.frq | wc -l

We can remove them with the plink option `--maf`.

In [ ]:
plink --bfile non_imputed --maf 0.05 --make-bed --out non_imputed_maf05

## Evaluating the level of missingness in our dataset

In [ ]:
plink --bfile non_imputed --missing --out missingness_check

PLINK will create two output, with thr prefix 'missingness_check'. 
* missingness_check.imiss
* missingness_check.lmiss

We can check the first ten lines of files contain with the command `head`:

### missingness_check.imiss

In [ ]:
head missingness_check.imiss

**i**miss stantds for individuals missingness, the file summarizes the missing genotype rates per individual. 
* FID contains the sample cluster/family name
* IID contains the sample individual ID
* MISS_PHENO indicates whether the phenotype information is missing (Y/N)
* N_MISS	Number of missing genotype call(s)
* N_GENO	Number of potentially valid call(s)
* F_MISS	Missing call rate, proportion of N_MISS/N_GENO

We can check the 20 samples with:
1) highest missingness
2) lowest missingness

In [ ]:
sort -k6,6 -n missingness_check.imiss | tail -n 20
sort -k6,6 -nr missingness_check.imiss | tail -n 20

#### Removing samples with high missingness
For filtering out low-quality samples we can use the PLINK option `--mind N`. With this option we will remove all samples with a missingess equal or higher than N. 

In [ ]:
plink --bfile non_imputed --mind 0.9 --make-bed --out non_imputed_mind9 

If we use N = 0.9, how many samples will be removed?

### missingness_check.lmiss

In [ ]:
head missingness_check.lmiss

**l**miss stands for locus missingness, it summarizes the missing genotype rates per SNP (locus).
* CHR column inform on the chromosome number.
* SNP is SNP ID.
* N_MISS Number of missing genotypes for this SNP across all individuals.
* N_GENO Total number of genotypes for this SNP across all individuals.
* F_MISS Proportion of missing genotypes for this SNP (N_MISS / N_GENO).

#### Removing SNPs with high missingness
For filtering out low-quality SNPs we can use the PLINK option `--geno N`. With this option we will remove all variants with a missingess equal or higher than N. 

In [ ]:
plink --bfile non_imputed --geno 0.9 --make-bed --out non_imputed_geno9 

## Linkage Disequilibrium

Linkage disequilibrium refers to the non-random association of alleles at different loci. When two SNPs are in LD, they tend to be inherited together more often than would be expected by chance. \
When not accounting for LD, we mistakenly interpret an association between two variants as causal, when in reality the association could be due to their co-inheritance.

To remove sites in LD, we can use PLINK option `--indep-pairwise`. It takes 3 parameters: window size, step size, r^2 threshold.

1) We select a window size in variant count (i.e. 100, will select a window with 100 SNPs, first parameter)
2) We remove all SNPs with a r^2 equal or higher than our threshold (i.e. 0.1, third parameter), measure the association between two SNPs
3) PLINK will shift the window a number of variants, and perform step 1 and 2 (i.e. 10, will shift the window 10 SNPs, second parameter)


In [ ]:
--indep-pairwise <window size>['kb'] <step size (variant ct)> <r^2 threshold>

In [ ]:
plink --bfile non_imputed --indep-pairphase 1000 100 0.5 --out pruning_report

This command will output two files: pruning_report.prune.in and pruning_report.prune.out
* prune.in, variants that passed the pruning parameters
* prune.out, variants that did NOT pass the pruning parameters

We can now subset the dataset based on the pruning_report files, by running **one** of these two line of code.

In [ ]:
plink --bfile non_imputed --extract pruning_report.prune.in --make-bed --out non_imputed_pruned 
plink --bfile non_imputed --exclude pruning_report.prune.out --make-bed --out non_imputed_pruned 

# Visualizing the genetic variability with PCA

We are providing you a script that will perform the following:
1) Convert vcf to plink format
2) Apply maf and LD filters
3) Convert plink file to eigenstrat format
4) Run smartpca

In [ ]:
python /gpfs/helios/projects/echo_workshops/project.1.tk/scripts/VCF2smartpca.py
--name my_pca_script 
--input_file non_imputed 
--maf 0.05 
--pruning '1000 100 0.5' 
--pca_project YES 
--pca_controls Albanian.HO,Belarusian.HO,Bulgarian.HO,Croatian.HO,Cypriot.HO,Czech.HO,English.HO,Estonian.HO,Finnish.HO,French.HO,Greek.HO,Hungarian.HO,Icelandic.HO,Italian.North.HO,Lithuanian.HO,Maltese.HO,Norwegian.HO,Orcadian.HO,Romanian.HO,Russian.HO,Scottish.HO,Sicilian.HO,Spanish.HO,Spanish.North.HO,Ukrainian.HO

The script will create a "my_pca_script.sh" bash file with all command lines needed

In [ ]:
more my_pca_script.sh

In [ ]:
sbatch my_pca_script.sh

Once the script is done running, you will find in your directory two files:
* **pca.evec**, containing the coordinates of each individual along each principal component (eigenvectors).
* **eval**, containing the eigenvalues, indicating how much variance each principal component explains.

#### Calculate the variance explained for each PC

In [ ]:
awk '{sum += $1} END {print sum}' non_imputed_maf0.05_pruned.eval

Let's say it returns 108

In [ ]:
cat non_imputed_maf0.05_pruned.eval | while read line; 
do explained_variance=$(echo "scale=4; $line / 108 * 100" | bc); 
echo "Explained variance: $explained_variance%" >> explained_variance_output.txt; done

#### You can plot the PCA using PCA_plot.R script

In [ ]:
conda activate echo_workshop

In [ ]:
Rscript /gpfs/helios/projects/echo_workshops/project.1.tk/scripts/PCA_plot.R non_imputed_maf0.05_pruned.pca.evec output.pdf 
# or
Rscript /gpfs/helios/projects/echo_workshops/project.1.tk/scripts/PCA_plot.R non_imputed_maf0.05_pruned.pca.evec output_focus.pdf --focus FranceLIA.Anc,
KoksijdeEMA.Anc

### ggrepel error?

Try this:

In [ ]:
R

In [ ]:
install.packages("ggrepel")

And follow the instructions

In [ ]:
install.packages("dplyr")

And follow the instructions

In [ ]:
install.packages("optparse")

And follow the instructions

if python issues are thrown: try pip install packaging==21.0

#### A few notes on the smartpca par file

Here you can find additional info: https://github.com/chrchang/eigensoft/blob/master/POPGEN/README

This is the par file (therefore, the options) we used for our PCA:

In [ ]:
genotypename: {output_name}.geno
snpname: {output_name}.snp
indivname: {output_name}.ind
evecoutname: {output_name}.pca.evec
evaloutname: {output_name}.eval
numoutevec: 4

- numoutvec: number of eigenvectors to output. It's the number of PCs. While the default is 10, we used only 4.

##### Projection 
- lsqproject:  YES PCA projections is carried out by solving least squares equations rather than an orthogonal projection step.  This is approriate if PCs are calculated using samples with little missing data but it is desired to project samples with much missing data onto the top PCs.
- poplistname:   If wishing to infer eigenvectors using only individuals from a subset of populations, and then project individuals from all populations onto those eigenvectors, this input file contains a list of population names, one population name per line, which will be used to infer eigenvectors. It is assumed that the population of each individual is specified in the indiv file. Default is to use individuals from all populations.

##### Manipulating the normalization
- altnormstyle: Affects very subtle details in normalization formula. Default is YES (normalization formulas of Patterson et al. 2006). To match EIGENSTRAT (normalization formulas of Price et al. 2006), set to NO.

##### Removing Outliers
- numoutlieriter: maximum number of outlier removal iterations. Default is 5.To turn off outlier removal, set this parameter to 0.
- numoutlierevec: number of principal components along which to remove outliers during each outlier removal iteration.  Default is 10.
- outliersigmathresh: number of standard deviations which an individual must exceed, along one of the top (numoutlierevec) principal components, in order for that individual to be removed as an outlier.  Default is 6.0.

##### Shrink Mode
- shrinkmode: YES  (default NO) 
A problem with smartpca is that samples used to calculate the PC axes 
"stretch" the axes.  So that 2 populations in fact genetically identical 
(2 independent samples from the same underlying population) will appear different 
if one is used to compute axes, and one not.  shrinkmode: YES is an attempt 
to solve this problem.  
*** warning *** shrinkmode is slow and will greatly increase the runtime. 
(NEW) New version:  newshrink:  YES technical variation of shrinkmode,  should be (slightly) 
- autoshrink: YES  (default NO)  A new method (Liu, Dobriban, Singer: https://arxiv.org/pdf/1611.05550.pdf) of eigenvalue shrinkage. This costs little in CPU, but the results seem slightly worse than shrinkmode: YES.  

##### Fst Estimation
- fstonly: If set to YES, then skip PCA and just calculate FST values. The default value for this parameter is NO.